In [1]:
!pip install -U langgraph -q
!pip install langchain -q
!pip install ollama -q

!pip install pymongo -q

In [2]:
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, MessageGraph
from langchain_community.llms import Ollama
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
database = client['LangChain_Intento4']
collection = database['lenguajes']

In [4]:
# aqui debe llegar user_name y user_email
# esto buscará el usuario según el email

def function1(state): #buscar usuario
    messages = state['messages']
    agent_response = messages[-1]
    documents = collection.find({'email': user_email})
    user_agenda = []
    for document in documents:
        user_agenda.append(document)
        state['messages'].append(user_agenda)
        return state

In [5]:
user_dia = "15"
user_mes = "mayo"
user_hora = "14"

# aqui debe llegar user_dia, user_mes y user_hora

def function2(state): #agendar cita

    messages = state['messages']

    user_cita = user_dia + " de " + user_mes + " del 2024 a las " + user_hora + " horas"
    agenda_local = collection.find({'cita': user_cita})
    repetidas = []

    for agenda_citas in agenda_local:
        repetidas.append(agenda_citas)
    if len(repetidas)>0:
        print("agenda otra hora")
    else:
        print("añadir cita")
    

In [ ]:
llm1 = Ollama(model='llama3', temperature=0)

In [ ]:
prompt = "Cuanto es 1 + 1?"

In [8]:
graph = MessageGraph()

graph.add_node("oracle", llm1)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

In [16]:
terminar = "bye"
user_input = ""
store = []
inicializador = "eres un asistente chatbot que habla español"
store.append("System: " + inicializador)
iniciar = runnable.invoke(HumanMessage(inicializador))
store.append("AI: " + iniciar[-1].content)

while user_input != terminar:
    user_input = input("Ingrese mensaje: ")
    store.append("Human: " + user_input)
    respuesta = runnable.invoke(HumanMessage(str(store)))
    store.append(respuesta[-1].content)
    print(store)

Ingrese mensaje:  Hola, me llamo Osvaldo


['System: eres un asistente chatbot que habla español', 'AI: Hola! Soy un chatbot que puede ayudarte en español. ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta o necesitas ayuda con algo en particular? Estoy aquí para escucharte y ayudarte lo mejor que pueda. ¡Hablemos!', 'Human: Hola, me llamo Osvaldo', 'AI: ¡Hola Osvaldo! Me alegra poder hablar contigo. ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta o necesitas ayuda con algo en particular? Estoy aquí para escucharte y ayudarte lo mejor que pueda. ¡Hablemos!']


Ingrese mensaje:  cómo me llamo?


['System: eres un asistente chatbot que habla español', 'AI: Hola! Soy un chatbot que puede ayudarte en español. ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta o necesitas ayuda con algo en particular? Estoy aquí para escucharte y ayudarte lo mejor que pueda. ¡Hablemos!', 'Human: Hola, me llamo Osvaldo', 'AI: ¡Hola Osvaldo! Me alegra poder hablar contigo. ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta o necesitas ayuda con algo en particular? Estoy aquí para escucharte y ayudarte lo mejor que pueda. ¡Hablemos!', 'Human: cómo me llamo?', "It seems like you're having a conversation with an AI chatbot in Spanish! Let's summarize what's been said so far:\n\n1. The AI introduces itself as a chatbot that can assist in Spanish.\n2. You, Osvaldo, greet the AI and ask how to refer to yourself.\n\nNow it's your turn again! What would you like to say next?"]


KeyboardInterrupt: Interrupted by user

## Conditional Edges

# Paso 1

In [ ]:
# asigna AgentState como un diccionario vacío
AgentState = {}

# La key de mensajes se asignará como un array vacío. Agregaremos nuevos mensajes a medida que pasemos por los nodos. 
AgentState["messages"] = []

In [ ]:
AgentState

##### Nuestro objetivo es que este estado se llene así: {'messages': [HumanMessage, AIMessage, ...]]}
Además, ahora necesitamos modificar nuestras funciones para pasar información a lo largo del nuevo AgentState.te

In [ ]:
def function1(state): #buscar usuario
    messages = state['messages']
    user_input = messages[-1]
    documents = str({
  "nombre": "Osvaldo",
  "correo": "osvaldo@email.com",
  "citacion": "miércoles 22 de mayo a las 15:00",
  "telefono": ""
})
    complete_query = "Dime los siguientes datos obtenidos desde una base de datos:" + documents
    respuesta = llm1.invoke(complete_query)
    print(respuesta)
    print(respuesta[-1])
    state['messages'].append(respuesta) # añadiendo la respuesta AIMessage al AgentState
    return state

In [ ]:
def function2(state): #agendar cita

    messages2 = state['messages']
    datos_recibidos = messages2[-1]
    query_nueva = "Tienes los siguientes datos: " + datos_recibidos + ", consulta por los faltantes, y si no faltan, pregunta si quiere agendar una cita nueva"
    respuesta_nueva = llm1.invoke(query_nueva)
    print(respuesta_nueva)
    print(respuesta_nueva[-1])
    state['messages'].append(respuesta_nueva)
    return state
    

In [ ]:
def function3(state): 
    messages3 = state['messages']
    datos_recibidos2 = messages3[-1]
    query_nueva2 = "Asume que la respuesta fue un sí y pregunta a qué hora la cita nueva"
    respuesta_nueva2 = llm1.invoke(query_nueva2)
    print(respuesta_nueva2)
    print(respuesta_nueva2[-1])
    state['messages'].append(respuesta_nueva2)
    return state


In [ ]:
from langgraph.graph import Graph

workflow = Graph()


workflow.add_node("agent", function1)
workflow.add_node("tool", function2)
workflow.add_node("responder", function3)

workflow.add_edge('agent', 'tool')
workflow.add_edge('tool', 'responder')

workflow.set_entry_point("agent")
workflow.set_finish_point("responder")

app = workflow.compile()

In [ ]:
inputs = {"messages": ["what is the temperature in las vegas"]}
app.invoke(inputs)

In [ ]:
input = {"messages": ["Quiero una cita, me llamo Osvaldo"]}
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Como notamos que se están agregando muchas cosas al array, podemos hacerlo un poco más fácil con lo siguiente:

```bash
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]seMessage], operator.add]


# Conditonal Edges

Podemos hacer que nuestro agente sea más inteligente diciéndole que solo use la herramienta cuando sea necesario, si no simplemente responda al usuario. 

La forma en que podemos hacer este LangGraph es:
1. Vincular una Tool al agente
2. Añadirr unaConditional Edgel al agente con la opción de llamar a laTool  o n
3. definiendo los criterios para el Conditional Edge sobre cuándo llamar a la Tool. Definiremos una función para esto..

Comencemos con la definición de AgentState como se menciona algunas celdas arriba..

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

La herramienta de vinculación con agente (modelo LLM) se simplifica en langchain

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function

model = Ollama(temperature=0, model='llama3')
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

model = Ollama(model='llama3',temperature=0)
model_with_tools = model.bind_tools([multiply])

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([multiply])
builder.add_node("multiply", tool_node)

builder.add_edge("multiply", END)

builder.set_entry_point("oracle")

# Intento de memoria

In [17]:
!pip install langchain -q

In [18]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [19]:
!pip install ollama -q

In [20]:
from langchain_community.llms import Ollama

model = Ollama(model="llama3", temperature=0)

In [21]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage("Hi! I'm Bob")])

"AI: Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [22]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

'Bob! Your name is indeed Bob.'

In [23]:
! pip install langchain_community -q

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [25]:
config = {"configurable": {"session_id": "abc2"}}

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"AI: Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"I'm happy to help! Since we just started our conversation, I don't have any information about your name yet. Would you like to introduce yourself and tell me what your name is?"